# Integrate Data Validation Into Your Pipeline



In [13]:
# Prep environment and logging

import json
import os
import logging
import great_expectations as ge
import great_expectations.jupyter_ux
import pandas as pd
import uuid # used to generate run_id
from datetime import datetime

import tzlocal

great_expectations.jupyter_ux.setup_notebook_logging()



2022-04-20T11:09:11+0200 - INFO - Great Expectations logging enabled at INFO level by JupyterUX module.
2022-04-20T11:09:11+0200 - INFO - Great Expectations logging enabled at INFO level by JupyterUX module.
2022-04-20T11:09:11+0200 - INFO - Great Expectations logging enabled at INFO level by JupyterUX module.


## Integrate data validation into your pipeline

[**Watch a short tutorial video**](https://docs.greatexpectations.io/en/latest/getting_started/pipeline_integration.html?utm_source=notebook&utm_medium=integrate_validation#video)


[**Read more in the tutorial**](https://docs.greatexpectations.io/en/latest/getting_started/pipeline_integration.html?utm_source=notebook&utm_medium=integrate_validation)

**Reach out for help on** [**Great Expectations Slack**](https://tinyurl.com/great-expectations-slack)




### Get a DataContext object


In [14]:
context = ge.data_context.DataContext()

### Get a pipeline run id

[Read more in the tutorial](https://docs.greatexpectations.io/en/latest/getting_started/pipeline_integration.html?utm_source=notebook&utm_medium=integrate_validation#set-a-run-id)


In [15]:
# Generate a run-id that GE will use to key shared parameters
run_id = datetime.utcnow().isoformat().replace(":", "") + "Z"
run_id

'2022-04-20T090912.720130Z'

### Choose data asset name and expectation suite name

[Read more in the tutorial](https://docs.greatexpectations.io/en/latest/getting_started/pipeline_integration.html?utm_source=notebook&utm_medium=integrate_validation#choose-data-asset-and-expectation-suite)


In [16]:
context.list_expectation_suites()

{'data__dir': {'default': {'taiwanese-bankruptcy': ['my_suite',
    'BasicDatasetProfiler']}}}

In [17]:
data_asset_name = "taiwanese-bankruptcy" # TODO: replace with your value!
expectation_suite_name = "my_suite" # TODO: replace with your value!

### Obtain the batch to validate

[Read more in the tutorial](https://docs.greatexpectations.io/en/latest/getting_started/pipeline_integration.html?utm_source=notebook&utm_medium=integrate_validation#obtain-a-batch-to-validate)



##### If your pipeline processes Pandas Dataframes:

```
df = pd.read_csv(file_path_to_validate)
df.head()
batch = context.get_batch(data_asset_name, expectation_suite_name, df)
```

##### If your pipeline processes Spark Dataframes:
```
from pyspark.sql import SparkSession
from great_expectations.dataset import PandasDataset, SqlAlchemyDataset, SparkDFDataset
spark = SparkSession.builder.getOrCreate()
df = SparkDFDataset(spark.read.csv(file_path_to_validate))
df.spark_df.show()
batch = context.get_batch(data_asset_name, expectation_suite_name, df)
```

##### If your pipeline processes SQL querues:
```
batch = context.get_batch(data_asset_name, expectation_suite_name, query="SELECT * from ....") # the query whose result set you want to validate
```


In [18]:
df = pd.read_csv('../../../00_Dask/data/taiwanese-bankruptcy.csv')
df.head()
batch = context.get_batch(data_asset_name, expectation_suite_name, df)


### Validate the batch

This is the "workhorse" method of Great Expectations. Call it in your pipeline code after loading the file and just before passing it to your computation.

[Read more in the tutorial](https://docs.greatexpectations.io/en/latest/getting_started/pipeline_integration.html?utm_source=notebook&utm_medium=integrate_validation#validate)



In [19]:
validation_result = batch.validate(run_id=run_id)

if validation_result["success"]:
    print("This file meets all expectations from a valid batch of {0:s}".format(data_asset_name))
else:
    print("This file is not a valid batch of {0:s}".format(data_asset_name))


2022-04-20T11:09:16+0200 - INFO - 74 expectation(s) included in expectation_suite.
2022-04-20T11:09:16+0200 - INFO - 74 expectation(s) included in expectation_suite.
2022-04-20T11:09:16+0200 - INFO - 74 expectation(s) included in expectation_suite.
This file meets all expectations from a valid batch of taiwanese-bankruptcy


### Review the validation results

[Read more in the tutorial](https://docs.greatexpectations.io/en/latest/getting_started/pipeline_integration.html?utm_source=notebook&utm_medium=integrate_validation#review-validation-results)


In [20]:
print(json.dumps(validation_result, indent=4))

{
    "results": [
        {
            "success": true,
            "result": {
                "element_count": 6819,
                "missing_count": 0,
                "missing_percent": 0.0,
                "unexpected_count": 0,
                "unexpected_percent": 0.0,
                "partial_unexpected_list": []
            },
            "exception_info": {
                "raised_exception": false,
                "exception_message": null,
                "exception_traceback": null
            },
            "expectation_config": {
                "expectation_type": "expect_column_values_to_not_be_null",
                "kwargs": {
                    "column": "Bankrupt?"
                }
            }
        },
        {
            "success": true,
            "result": {
                "observed_value": [
                    0,
                    1
                ],
                "element_count": 6819,
                "missing_count": 0,
                "miss

### Finishing touches - notifications and saving validation results and validated batches

#### Notifications
You want to be notified when the pipeline validated a batch, especially when the validation failed.

[Read more in the tutorial](https://docs.greatexpectations.io/en/latest/getting_started/pipeline_integration.html?utm_source=notebook&utm_medium=integrate_validation#send-notifications)

#### Saving validation results

To enable the storing of validation results, uncomment the `result_store` section in your great_expectations.yml file. 

[Read more in the tutorial](https://docs.greatexpectations.io/en/latest/getting_started/pipeline_integration.html?utm_source=notebook&utm_medium=integrate_validation#save-validation-results)

#### Saving failed batches

When a batch fails validation (it does not pass all the expectations of the data asset), it is useful to save the batch along with the validation results for future review. You can enable this option in your project's great_expectations.yml file. 

[Read more in the tutorial](https://docs.greatexpectations.io/en/latest/getting_started/pipeline_integration.html?utm_source=notebook&utm_medium=integrate_validation#save-failed-batches)

